In [21]:
import torch
import torch.nn as nn

# Importamos dataset y dataloader, para procesar los datos para ingresarlos a los modelos
# Nos permite usar nuestros datos para generar nuestro mdoelo de entrenamiento, agrupa los ejemplos con sus respectivas etiquetas.
# Dataloader, convierte dataset a un objeto iterable para pytorch, nos permite realizar nuestros propios minibatches de forma automatica 
# De forma simple con dataloader
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import numpy as np
import nltk
nltk.download('punkt')
import random
import json
import string
import time


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\feden\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### DEFINE NEURAL NETWORK

In [22]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, hidden_size) 
        self.l4 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        # no activation and no softmax at the end
        return out
    


### Methods
    For clean data
    Tokenize sentence
    Bag of Words

In [23]:

#REMOVE ACCENTS
def removeAccents(word):
    translation = str.maketrans("áéíóúüàèìùò", "aeiouuaeiou")
    return word.translate(translation)


#REMOVE PUNCTUATION AND SINGS
def removePunctuation (word):
    word = word.translate(str.maketrans('', '', string.punctuation))   
    return word


#TOKENIZE WORDS
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

# BAG OF WORDS
def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """

    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in tokenized_sentence: 
            bag[idx] = 1

    return bag



### LOAD DATA

In [30]:

with open('chitchat_intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
        tag = intent['tag']
        
        # add to tag list
        tags.append(tag)
        for pattern in intent['patterns']:
            
            #Clean data
            # tokenize each word in the sentence
            words = tokenize(pattern)
            
            # Remove accents
            w = [removeAccents(word) for word in words]

            # Remove punctuation
            w = [removePunctuation(word) for word in words]

            # Remove empty slots    
            w = [word for word in words if word != '']
            
            # add to our words list
            all_words.extend(words)
            
            # add to xy pair
            xy.append((words, tag))

# Remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))


print(f"{len(xy)} patterns: {xy}\n")
# print(len(tags), "tags:", tags)
# print("")
# print(len(all_words), " words:", all_words)


56 patterns: [(['hola'], 'CC_HOLA'), (['hi'], 'CC_HOLA'), (['hello'], 'CC_HOLA'), (['ciao'], 'CC_HOLA'), (['buen', 'dia'], 'CC_HOLA'), (['buenas', 'tardes'], 'CC_HOLA'), (['buenas', 'noches'], 'CC_HOLA'), (['hey'], 'CC_HOLA'), (['whats', 'up'], 'CC_HOLA'), (['que', 'onda'], 'CC_HOLA'), (['hola', 'que', 'tal'], 'CC_HOLA'), (['hola', 'amigo'], 'CC_HOLA'), (['hola', 'holaa'], 'CC_HOLA'), (['cya'], 'CC_ADIOS'), (['nos', 'vemos'], 'CC_ADIOS'), (['Goodbye'], 'CC_ADIOS'), (['chau'], 'CC_ADIOS'), (['adios'], 'CC_ADIOS'), (['hasta', 'luego'], 'CC_ADIOS'), (['me', 'voy'], 'CC_ADIOS'), (['Cuantos', 'anios'], 'CC_EDAD'), (['Cuantos', 'aÃ±os', 'tenes'], 'CC_EDAD'), (['cual', 'es', 'tu', 'edad'], 'CC_EDAD'), (['que', 'tan', 'viejo', 'eres'], 'CC_EDAD'), (['edad', '?'], 'CC_EDAD'), (['tu', 'edad'], 'CC_EDAD'), (['dime', 'tus', 'aÃ±os'], 'CC_EDAD'), (['como', 'te', 'apodaron'], 'CC_NAME_BOT'), (['como', 'te', 'llamas'], 'CC_NAME_BOT'), (['cual', 'es', 'tu', 'nombre'], 'CC_NAME_BOT'), (['como', 'te', '

### Prepare Data for model
    80% of the dataset will be for train the model
    20% of the dataset will be for testing the model
    We can use random_split from Pytorch or Split the data manually

In [31]:
# number_rows = len(xy)    # The size of our dataset or the number of rows in excel table.  
# test_split = int(number_rows*0.3)  
# validate_split = int(number_rows*0.2) 
# train_split = number_rows - test_split - validate_split     
# train_set, validate_set, test_set = random_split(xy, [train_split, validate_split, test_split])   


# Create testing data
print("\n---------------CREATING TESTING DATA--------------------\n")

# Data for testing the model
test_array = []
percentage = int(len(xy)*0.2)
dataset_copy = xy

for i in range(percentage):
    item = dataset_copy.pop(random.randrange(len(xy)))
    test_array.append(item)
#     item = xy.pop(random.randrange(len(xy)))
#     bag = bag_of_words(item[0], all_words)
#     x_test.append(bag)
#     label = tags.index(item[1])
#     y_test.append(label)


# Create training data
print("\n---------------CREATING TRAINING DATA--------------------\n")

# Data for training
x_train = []
y_train = []

for (pattern_sentence, tag) in dataset_copy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    x_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)


x_train = np.array(x_train)
y_train = np.array(y_train)

# x_test = np.array(x_test)
# y_test = np.array(y_test)


---------------CREATING TESTING DATA--------------------


---------------CREATING TRAINING DATA--------------------



### TRAINING THE MODEL
    Define Hyper Parameters
        Epochs,
        Batch_size,
        Learning_rate,
        input, hidden, output sizes

In [26]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(x_train[0])
hidden_size = 8
output_size = len(tags)

# Train Class Dataset
class ChatTrainDataset(Dataset):

    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples


# Define Train Loader
train_set = ChatTrainDataset()
train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True, num_workers=0)

# Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create neural network
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
print("\n--------------- TRAINING DATA --------------------\n")
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
            words = words.to(device)
            labels = labels.to(dtype=torch.long).to(device)

            # Forward pass
            outputs = model(words)

            # if y would be one-hot, we must apply
            # labels = torch.max(labels, 1)[1]
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    if (epoch+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'\nFinal loss: {loss.item():.4f}\n')

trained_data = {
        "model_state": model.state_dict(),
        "input_size": input_size,
        "hidden_size": hidden_size,
        "output_size": output_size,
        "all_words": all_words,
        "tags": tags
}

FILE = "data.pth"
torch.save(trained_data, FILE)

print(f'training complete. file saved to {FILE}')


--------------- TRAINING DATA --------------------

Epoch [100/1000], Loss: 0.2520
Epoch [200/1000], Loss: 0.0080
Epoch [300/1000], Loss: 0.0022
Epoch [400/1000], Loss: 0.0014
Epoch [500/1000], Loss: 0.0012
Epoch [600/1000], Loss: 0.0003
Epoch [700/1000], Loss: 0.0002
Epoch [800/1000], Loss: 0.0001
Epoch [900/1000], Loss: 0.0001
Epoch [1000/1000], Loss: 0.0000

Final loss: 0.0000

training complete. file saved to data.pth


### TEST MODEL

In [32]:
total = 0
correcta = 0
with torch.no_grad():
    for item in test_array:

        #Clean data
        # tokenize each word in the sentence
        pattern = item[0]
        
        # Remove accents
        pattern = [removeAccents(word) for word in pattern]

        # Remove punctuation
        pattern = [removePunctuation(word) for word in pattern]

        # Remove empty slots    
        pattern = [word for word in pattern if word != '']
            

        X = bag_of_words(pattern, all_words)
        X = X.reshape(1, X.shape[0])
        X = torch.from_numpy(X).to(device)

        output = model(X)
        _, predicted = torch.max(output, dim=1)
        
        tag = tags[predicted.item()]

        probs = torch.softmax(output, dim=1)
        prob = probs[0][predicted.item()]
        
        print(f"EJEMPLO: {item[0]} - Intencion:{item[1]}")
        print(f"PREDICCION del Ejemplo: {item[0]} = Intencion:{tag} con Confianza:{prob.item()} \n")
        total += prob.item()
        if(tag == item[1]):
            correcta += 1
            
print("Accuracy de tu modelo:",total / len(test_array))      
print(f"Clasifico correctamente el {int(correcta/0.1)}% de los ejemplos para testing")

EJEMPLO: ['cual', 'es', 'tu', 'funcion'] - Intencion:CC_PARA_QUE_FUE_CREADO
PREDICCION del Ejemplo: ['cual', 'es', 'tu', 'funcion'] = Intencion:CC_PARA_QUE_FUE_CREADO con Confianza:0.9999262094497681 

EJEMPLO: ['hola', 'amigo'] - Intencion:CC_HOLA
PREDICCION del Ejemplo: ['hola', 'amigo'] = Intencion:CC_HOLA con Confianza:0.9999123811721802 

EJEMPLO: ['estas', 'chevere'] - Intencion:CC_COMO_ESTAS
PREDICCION del Ejemplo: ['estas', 'chevere'] = Intencion:CC_COMO_ESTAS con Confianza:0.9999265670776367 

EJEMPLO: ['buenas', 'noches'] - Intencion:CC_HOLA
PREDICCION del Ejemplo: ['buenas', 'noches'] = Intencion:CC_HOLA con Confianza:0.9999912977218628 

EJEMPLO: ['con', 'que', 'me', 'puedes', 'ayudar'] - Intencion:CC_PARA_QUE_FUE_CREADO
PREDICCION del Ejemplo: ['con', 'que', 'me', 'puedes', 'ayudar'] = Intencion:CC_EL_CLIMA con Confianza:0.9794712662696838 

EJEMPLO: ['que', 'temperatura', 'hay', 'aqui'] - Intencion:CC_EL_CLIMA
PREDICCION del Ejemplo: ['que', 'temperatura', 'hay', 'aqui'] 

### Accuracy

In [12]:
# loop thru each data item
#     get item predictor input values
#     get item target values (0 or 1 or 2 . . )
#     use inputs to compute output logit values 
    
#     find index of largest output logit value
#     if index == target
#       correct prediction
#     else
#       wrong prediction
#   end-loop
#   return num correct / (num correct + num wrong)

plt.plot(train_accu,'-o')
plt.plot(eval_accu,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Accuracy')
 
plt.show()

NameError: name 'plt' is not defined

### CHAT

In [13]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('chitchat_intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "WAMAC"
print("Hola! hablemos un rato... (type 'quit' to exit)")
while True:
    sentence = input("Tu: ")

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    print(f"Confianza:{prob.item()} -> Intencion:{tag}")

    if prob.item() > 0.7:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
        if tag == "CC_ADIOS":
            break
    else:
        print(f"{bot_name}: No te entendi...")

Hola! hablemos un rato... (type 'quit' to exit)
Tu: hola
Confianza:0.9999517202377319 -> Intencion:CC_HOLA
WAMAC: Hola! Como puedo ayudarte?
Tu: nose, todo bien ?
Confianza:0.7963751554489136 -> Intencion:CC_HOLA
WAMAC: Hola! Como puedo ayudarte?
Tu: como estas
Confianza:0.999970555305481 -> Intencion:CC_COMO_ESTAS
WAMAC: Me siento muy bien!
Tu: como te llamas
Confianza:0.9133269786834717 -> Intencion:CC_NAME_BOT
WAMAC: No se si ya me pusieron mi nombre, pero escuche rumores de que me llamo WAMAC.
Tu: cual es tu nombre
Confianza:0.9999804496765137 -> Intencion:CC_NAME_BOT
WAMAC: No se si ya me pusieron mi nombre, pero escuche rumores de que me llamo WAMAC.
Tu: nombre
Confianza:0.5256704092025757 -> Intencion:CC_COMO_ESTAS
WAMAC: No te entendi...
Tu: nombre
Confianza:0.5256704092025757 -> Intencion:CC_COMO_ESTAS
WAMAC: No te entendi...
Tu: nombre tu
Confianza:0.9988136291503906 -> Intencion:CC_NAME_BOT
WAMAC: No se si ya me pusieron mi nombre, pero escuche rumores de que me llamo WAMAC.

KeyboardInterrupt: Interrupted by user